# Agents

```{include} ../warning.md
```

AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages.
All agents share the following attributes and methods:

- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.name`: The unique name of the agent.
- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.description`: The description of the agent in text.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`: Send the agent a sequence of {py:class}`~autogen_agentchat.messages.ChatMessage` get a {py:class}`~autogen_agentchat.base.Response`.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream`: Same as {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages` but returns an iterator of {py:class}`~autogen_agentchat.messages.AgentMessage` followed by a {py:class}`~autogen_agentchat.base.Response` as the last item.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_reset`: Reset the agent to its initial state.

See {py:mod}`autogen_agentchat.messages` for more information on AgentChat message types.


## Assistant Agent

{py:class}`~autogen_agentchat.agents.AssistantAgent` is a built-in agent that
uses a language model with ability to use tools.

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models import OpenAIChatCompletionClient


# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

We can call the {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages` 
method to get the agent to respond to a message.

In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=15), content=[FunctionCall(id='call_hqVC7UJUPhKaiJwgVKkg66ak', arguments='{"query":"AutoGen"}', name='web_search')]), ToolCallResultMessage(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_hqVC7UJUPhKaiJwgVKkg66ak')])]
source='assistant' models_usage=RequestUsage(prompt_tokens=92, completion_tokens=14) content='AutoGen is a programming framework designed for building multi-agent applications.'


The call to the {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages` method
returns a {py:class}`~autogen_agentchat.base.Response`
that contains the agent's final response in the {py:attr}`~autogen_agentchat.base.Response.chat_message` attribute,
as well as a list of inner messages in the {py:attr}`~autogen_agentchat.base.Response.inner_messages` attribute,
which stores the agent's "thought process" that led to the final response.

### Stream Messages

We can also stream each message as it is generated by the agent by using the
{py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages_stream` method,
and use {py:class}`~autogen_agentchat.ui.Console` to print the messages
as they appear to the console.

In [ ]:
from autogen_agentchat.ui import Console


async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream.
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Find information on AutoGen", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Find information on AutoGen", source="user")],
            cancellation_token=CancellationToken(),
        )
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

---------- assistant ----------
[FunctionCall(id='call_fSp5iTGVm2FKw5NIvfECSqNd', arguments='{"query":"AutoGen information"}', name='web_search')]
[Prompt tokens: 61, Completion tokens: 16]
---------- assistant ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_fSp5iTGVm2FKw5NIvfECSqNd')]
---------- assistant ----------
AutoGen is a programming framework designed for building multi-agent applications. If you need more detailed information or specific aspects about AutoGen, feel free to ask!
[Prompt tokens: 93, Completion tokens: 32]
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 154
Total completion tokens: 48
Duration: 4.30 seconds


The {py:meth}`~autogen_agentchat.agents.AssistantAgent.on_messages_stream` method
returns an asynchronous generator that yields each inner message generated by the agent,
and the last item is the final response message in the {py:attr}`~autogen_agentchat.base.Response.chat_message` attribute.

From the messages, you can see the assistant agent used the `web_search` tool to
search for information and responded using the search results.

### Understanding Tool Calling

Large Language Models (LLMs) are typically limited to generating text or code responses. However, many complex tasks benefit from the ability to use external tools that perform specific actions, such as fetching data from APIs or databases.

To address this limitation, modern LLMs can now accept a list of available tool schemas (descriptions of tools and their arguments) and generate a tool call message. This capability is known as **Tool Calling** or **Function Calling** and is becoming a popular pattern in building intelligent agent-based applications.

For more information on tool calling, refer to the documentation from [OpenAI](https://platform.openai.com/docs/guides/function-calling) and [Anthropic](https://docs.anthropic.com/en/docs/build-with-claude/tool-use).

## Other Preset Agents

The following preset agents are available:

- {py:class}`~autogen_agentchat.agents.CodeExecutorAgent`: An agent that can execute code.
- {py:class}`~autogen_ext.agents.OpenAIAssistantAgent`: An agent that is backed by an OpenAI Assistant, with ability to use custom tools.
- {py:class}`~autogen_ext.agents.MultimodalWebSurfer`: A multi-modal agent that can search the web and visit web pages for information.

## Next Step

Now we have discussed how to use the {py:class}`~autogen_agentchat.agents.AssistantAgent`,
we can move on to the next section to learn how to use the teams feature of AgentChat.

<!-- ## CodingAssistantAgent

Generates responses (text and code) using an LLM upon receipt of a message. It takes a `system_message` argument that defines or sets the tone for how the agent's LLM should respond. 

```python

writing_assistant_agent = CodingAssistantAgent(
    name="writing_assistant_agent",
    system_message="You are a helpful assistant that solve tasks by generating text responses and code.",
    model_client=model_client,
)
`

We can explore or test the behavior of the agent by sending a message to it using the  {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`  method. 

```python
result = await writing_assistant_agent.on_messages(
    messages=[
        TextMessage(content="What is the weather right now in France?", source="user"),
    ],
    cancellation_token=CancellationToken(),
)
print(result) -->